# Minimum Cost Diet Problem: Group Casimir Funk

## Topic: Vegan Teacher vs Liver King: Comparing Food Populations In Berkeley

## Objectives:

* Develop code to find the minimum pricing for maximum nutrition content for all three diets

* Compare feasibility, health and pricing of all three diets in Berkeley, California

* Create recipes that fulfill the minimum nutrient requirements for each diet and compare their pricing

## Table of Contents:
* [[A] Description of population of interest](#description)
* [[A] Dietary Reference Intakes ](#dri)
* [[A] Data on prices for different foods](#prices)
* [[A] Nutritional content of different foods](#nutrition)
* [[A] Solution](#solution)
* [[C] Sensitivity of Solution](#sensitivity)

#### Setup

In [93]:
!pip install -r requirements.txt --upgrade

import pandas as pd
from eep153_tools.sheets import read_sheets
import fooddatacentral as fdc
import warnings
from  scipy.optimize import linprog as lp
import numpy as np
import warnings
import cufflinks as cf

In [94]:
# API key; substitute your own!
apikey = "Hfo8BxP2zgYjatcMv4zuDmmCkzhiwzWWNJYE3iOQ"

## [A] Description of population of interest <a class="anchor" id="description"></a>

We’ve decided to study the minimum viable diet of raw vegans in comparison to raw meat diets and how both compare in price to the minimum for the USDA recommended diet. Our population of interests are students at UC Berkeley, which we haved generalized in our project to being ages 19-30. 

## [A] Dietary Reference Intakes  <a class="anchor" id="dri"></a>

minimum_nutrients() and  maximum_nutrients() takes as arguments the characteristics of a
person (e.g., age, sex) and returns a pandas.Series of Dietary Reference Intakes (DRI's) of a variety of nutrients appropriate for your population of interest.

In [95]:
def minimum_nutrients(age, sex):
    ntmin = pd.read_csv('./diet_minimums.csv').set_index('Nutrition').iloc[:,2:]
    headers = ntmin.columns.values
    sex = sex.lower()
    male = lambda x:x if 'M' in x or 'C' in x else None
    female = lambda x:x if 'F' in x or 'C' in x else None
    if sex == 'male' or sex == 'm':
        filtered = [male(x) for x in headers]
        temp = [x for x in filtered if x is not None]
    elif sex == 'female' or sex == 'f':
        filtered = [female(x) for x in headers]
        temp = [x for x in filtered if x is not None]
    if age < 4:
        return ntmin[temp[0]]
    elif age < 9:
        return ntmin[temp[1]]
    elif age < 14:
        return ntmin[temp[2]]
    elif age < 19:
        return ntmin[temp[3]]
    elif age < 31:
        return ntmin[temp[4]]
    elif age < 51: 
        return ntmin[temp[5]]
    else:
        return ntmin[temp[6]]

In [96]:
def maximum_nutrients(age, sex):
    ntmax = pd.read_csv('./diet_maximums.csv').set_index('Nutrition').iloc[:,2:]
    headers = ntmax.columns.values
    sex = sex.lower()
    male = lambda x:x if 'M' in x or 'C' in x else None
    female = lambda x:x if 'F' in x or 'C' in x else None
    if sex == 'male' or sex == 'm':
        filtered = [male(x) for x in headers]
        temp = [x for x in filtered if x is not None]
    elif sex == 'female' or sex == 'f':
        filtered = [female(x) for x in headers]
        temp = [x for x in filtered if x is not None]
    if age < 4:
        return ntmax[temp[0]]
    elif age < 9:
        return ntmax[temp[1]]
    elif age < 14:
        return ntmax[temp[2]]
    elif age < 19:
        return ntmax[temp[3]]
    elif age < 31:
        return ntmax[temp[4]]
    elif age < 51: 
        return ntmin[temp[5]]
    else:
        return ntmin[temp[6]]

In [97]:
minimum_nutrients(21,'m')

Nutrition
Energy                            2400.0
Protein                             56.0
Fiber, total dietary                33.6
Folate, DFE                        400.0
Calcium, Ca                       1000.0
Carbohydrate, by difference        130.0
Iron, Fe                             8.0
Magnesium, Mg                      400.0
Niacin                              16.0
Phosphorus, P                      700.0
Potassium, K                      4700.0
Riboflavin                           1.3
Thiamin                              1.2
Vitamin A, RAE                     900.0
Vitamin B-12                         2.4
Vitamin B-6                          1.3
Vitamin C, total ascorbic acid      90.0
Vitamin E (alpha-tocopherol)        15.0
Vitamin K (phylloquinone)          120.0
Zinc, Zn                            11.0
Name: M 19-30, dtype: float64

In [98]:
maximum_nutrients(21,'m')

Nutrition
Sodium, Na    2300
Name: M 19-30, dtype: int64

## [A] Data on prices for different foods <a class="anchor" id="prices"></a>

Retrieves our google spreadsheet of different prices for both raw meat and raw vegan diets, and concatenates them into a Dataframe.

In [99]:
sheets = [("https://docs.google.com/spreadsheets/d/1GuSUsrGc5GX6hPerOCEJTDN22wVadD62NtyRLToH-Ho/edit#gid=0",
           "Ingredients")]

df = read_sheets(sheets[0][0])[sheets[0][1]]

Key available for students@eep153.iam.gserviceaccount.com.


### Raw Meat Diet prices

In [100]:
raw_meat_df = df.loc[df["RAW MEAT"] == "v"]
raw_meat_df

,Food,Quantity,Units,Price,Date,Location,FDC,RAW VEGAN,RAW MEAT
0,Beef,1.00,lb,12.99,2-22-2023,Wholefood,746760,,v
1,Pork,1.00,lb,9.99,2-26-2023,Wholefood,749420,,v
2,"Chicken, drumstick",1.00,lb,3.99,2-26-2023,Wholefood,331897,,v
3,Canned Tuna,5.00,oz,0.99,2-27-2023,Wholefood,334194,,v
4,Atlantic Salmon,1.00,lb,15.99,2-27-2023,Wholefood,175167,,v
5,Bison,16.00,oz,10.99,2-27-2023,Wholefood,175293,,v
6,Bacon,1.00,lb,7.99,2-26-2023,Wholefood,749420,,v
7,Eggs,270.00,oz,5.99,2-26-2023,Wholefood,748967,,v
8,Turkey,1.00,lb,12.99,2-26-2023,Wholefood,746785,,v
9,Sausage,1.00,lb,6.99,2-26-2023,Wholefood,746783,,v


### Raw Vegan Diet prices

In [101]:
raw_vegan_df = df.loc[df["RAW VEGAN"] == "v"]
raw_vegan_df

,Food,Quantity,Units,Price,Date,Location,FDC,RAW VEGAN,RAW MEAT
18,Russet Potato,1.0,lb,1.69,2-22-2023,https://www.wholefoodsmarket.com/product/produ...,170027,v,v
19,Pasta,16.0,oz,1.59,2-22-2023,https://www.wholefoodsmarket.com/product/365-b...,172014,v,v
20,"Lemon, raw",58.0,grams,0.99,2023-2-27,https://www.wholefoodsmarket.com/product/produ...,2344662,v,v
21,"Lime, raw",67.0,grams,0.69,2023-2-27,https://www.wholefoodsmarket.com/product/produ...,2344664,v,v
22,"Nuts, pecans, halves, raw",7.0,oz,11.79,2023-2-27,https://www.wholefoodsmarket.com/product/auror...,2346395,v,v
...,...,...,...,...,...,...,...,...,...
81,"Quinoa, fat added",16.0,oz,3.79,2023-2-27,https://www.safeway.com/shop/product-details.9...,2343885,v,
82,"Almond milk, unsweetened, plain, refrigerated",64.0,oz,3.49,2023-2-27,https://www.safeway.com/shop/product-details.9...,2340791,v,
83,"Oat milk, unsweetened, plain, refrigerated",86.0,oz,4.99,2023-2-27,https://www.safeway.com/shop/product-details.9...,602962,v,
84,"WHITE SLICED MUSHROOMS, WHITE",8.0,oz,2.99,2023-2-27,https://www.wholefoodsmarket.com/product/produ...,2378248,v,


### USDA Diet prices

In [102]:
usda_df = df[(df["RAW MEAT"] == "v") | (df["RAW VEGAN"] == "v")]
usda_df

,Food,Quantity,Units,Price,Date,Location,FDC,RAW VEGAN,RAW MEAT
0,Beef,1.0,lb,12.99,2-22-2023,Wholefood,746760,,v
1,Pork,1.0,lb,9.99,2-26-2023,Wholefood,749420,,v
2,"Chicken, drumstick",1.0,lb,3.99,2-26-2023,Wholefood,331897,,v
3,Canned Tuna,5.0,oz,0.99,2-27-2023,Wholefood,334194,,v
4,Atlantic Salmon,1.0,lb,15.99,2-27-2023,Wholefood,175167,,v
...,...,...,...,...,...,...,...,...,...
81,"Quinoa, fat added",16.0,oz,3.79,2023-2-27,https://www.safeway.com/shop/product-details.9...,2343885,v,
82,"Almond milk, unsweetened, plain, refrigerated",64.0,oz,3.49,2023-2-27,https://www.safeway.com/shop/product-details.9...,2340791,v,
83,"Oat milk, unsweetened, plain, refrigerated",86.0,oz,4.99,2023-2-27,https://www.safeway.com/shop/product-details.9...,602962,v,
84,"WHITE SLICED MUSHROOMS, WHITE",8.0,oz,2.99,2023-2-27,https://www.wholefoodsmarket.com/product/produ...,2378248,v,


## [A] Nutritional content of different foods <a class="anchor" id="nutrition"></a>

Now we have a list of foods with prices.  Do lookups on USDA database
to get nutritional information.

In [103]:
## This function utilizes the FDC API to do nutritional content lookups for every food in our Data Frame.
def getNutritionalContent(diet):
    
    diet = diet.lower()
    
    if diet == "meat":
        diet_df = raw_meat_df
    elif diet == "vegan":
        diet_df = raw_vegan_df
    elif diet == "usda":
        diet_df = usda_df
    else:
        print("Diet not found, please choose from meat, vegan, or usda.")
        return
    
    D = {}
    for food in  diet_df.Food.tolist():
        try:
            FDC = diet_df.loc[diet_df.Food==food,:].FDC.iloc[0]
            D[food] = fdc.nutrients(apikey,FDC).Quantity
        except AttributeError: 
            warnings.warn("Couldn't find FDC Code %s for food %s or server is overwhelmed. Try again later!" % (food,FDC))        

    return pd.DataFrame(D,dtype=float)

### Raw Meat Diet Nutritional content

In [104]:
Raw_Meat_FoodNutrients = getNutritionalContent("meat")
Raw_Meat_FoodNutrients

,Beef,Pork,"Chicken, drumstick",Canned Tuna,Atlantic Salmon,Bison,Bacon,Eggs,Turkey,Sausage,...,Russet Potato,Pasta,"Lemon, raw","Lime, raw","Nuts, pecans, halves, raw","Nuts, walnuts, English, halves, raw","Cashews, NFS","Pistachio nuts, NFS","Almond butter, creamy","Peanut butter, creamy"
25-hydroxycholecalciferol,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.560,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Alanine,NaN,NaN,NaN,NaN,1.271,1.348,NaN,0.667,NaN,0.94,...,NaN,0.195,NaN,NaN,0.4375,0.59,NaN,NaN,1.181,1.160
"Alcohol, ethyl",NaN,NaN,NaN,NaN,0.000,0.000,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN
Amino acids,NaN,NaN,NaN,NaN,0.000,0.000,NaN,0.000,NaN,0.00,...,0.0,0.000,NaN,NaN,0.0000,0.00,NaN,NaN,0.000,0.000
Arginine,NaN,NaN,NaN,NaN,1.221,1.377,NaN,0.787,NaN,1.03,...,NaN,0.211,NaN,NaN,1.3630,2.02,NaN,NaN,2.771,3.323
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
cis-Lutein/Zeaxanthin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,57.000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
cis-Lycopene,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
cis-beta-Carotene,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
trans-Lycopene,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Raw Vegan Diet Nutritional content

In [105]:
Raw_Vegan_FoodNutrients = getNutritionalContent("vegan")
Raw_Vegan_FoodNutrients

,Russet Potato,Pasta,"Lemon, raw","Lime, raw","Nuts, pecans, halves, raw","Nuts, walnuts, English, halves, raw","Cashews, NFS","Pistachio nuts, NFS","Almond butter, creamy","Peanut butter, creamy",...,"Blueberries, raw","Raspberries, raw","Mangos, raw",Chia seeds,Flax seeds,"Quinoa, fat added","Almond milk, unsweetened, plain, refrigerated","Oat milk, unsweetened, plain, refrigerated","WHITE SLICED MUSHROOMS, WHITE",Nutritional Yeast
Alanine,NaN,0.195,NaN,NaN,0.4375,0.590,NaN,NaN,1.181,1.160,...,NaN,NaN,0.082,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"Alcohol, ethyl",NaN,NaN,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN,...,NaN,NaN,0.000,0.0,0.0,0.0,0.0,NaN,NaN,NaN
Amino acids,0.00,0.000,NaN,NaN,0.0000,0.000,NaN,NaN,0.000,0.000,...,NaN,NaN,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Arginine,NaN,0.211,NaN,NaN,1.3630,2.020,NaN,NaN,2.771,3.323,...,NaN,NaN,0.031,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Ash,1.13,0.550,NaN,NaN,1.4390,1.641,NaN,NaN,3.185,2.771,...,0.2288,0.35,0.360,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
cis-Lutein/Zeaxanthin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
cis-Lycopene,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
cis-beta-Carotene,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
trans-Lycopene,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### USDA Diet Nutritional content

In [106]:
USDA_FoodNutrients = getNutritionalContent("usda")
USDA_FoodNutrients

,Beef,Pork,"Chicken, drumstick",Canned Tuna,Atlantic Salmon,Bison,Bacon,Eggs,Turkey,Sausage,...,"Blueberries, raw","Raspberries, raw","Mangos, raw",Chia seeds,Flax seeds,"Quinoa, fat added","Almond milk, unsweetened, plain, refrigerated","Oat milk, unsweetened, plain, refrigerated","WHITE SLICED MUSHROOMS, WHITE",Nutritional Yeast
25-hydroxycholecalciferol,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.560,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Alanine,NaN,NaN,NaN,NaN,1.271,1.348,NaN,0.667,NaN,0.94,...,NaN,NaN,0.082,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"Alcohol, ethyl",NaN,NaN,NaN,NaN,0.000,0.000,NaN,NaN,NaN,NaN,...,NaN,NaN,0.000,0.0,0.0,0.0,0.0,NaN,NaN,NaN
Amino acids,NaN,NaN,NaN,NaN,0.000,0.000,NaN,0.000,NaN,0.00,...,NaN,NaN,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Arginine,NaN,NaN,NaN,NaN,1.221,1.377,NaN,0.787,NaN,1.03,...,NaN,NaN,0.031,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
cis-Lutein/Zeaxanthin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,57.000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
cis-Lycopene,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
cis-beta-Carotene,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
trans-Lycopene,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## [A] Solution <a class="anchor" id="solution"></a>

What is the minimum cost diet for our population? How much does it cost, and of what does it consist? How
does it vary with age, sex, and level of activity?

### Units & Prices

In [107]:
### We use the `units` function to convert all foods to either deciliters or hectograms, to match FDC database
def getUnitsAndPrices(diet):
    
    diet = diet.lower()
    
    if diet == "meat":
        diet_df = raw_meat_df
    elif diet == "vegan":
        diet_df = raw_vegan_df
    elif diet == "usda":
        diet_df = usda_df
    else:
        print("Diet not found, please choose from meat, vegan, or usda.")
        return
    
    # Convert food quantities to FDC units
    diet_df['FDC Quantity'] = diet_df[['Quantity','Units']].T.apply(lambda x : fdc.units(x['Quantity'],x['Units']))

    # Now may want to filter df by time or place--need to get a unique set of food names.
    diet_df['FDC Price'] = diet_df['Price']/diet_df['FDC Quantity']

    diet_df.dropna(how='any') # Drop food with any missing data

    # To use minimum price observed
    diet_prices = diet_df.groupby('Food',sort=False)['FDC Price'].min()

    return diet_prices

### Raw Meat Diet Units & Prices

In [108]:
meat_prices = getUnitsAndPrices("meat")
meat_prices

/opt/conda/lib/python3.9/site-packages/pandas/core/dtypes/cast.py:1990: UnitStrippedWarning:

The unit of the quantity is stripped when downcasting to ndarray.



Food
Beef                                    2.8638047857815594 / hectogram
Pork                                    2.2024179992269266 / hectogram
Chicken, drumstick                      0.8796444261176615 / hectogram
Canned Tuna                              0.698424446601692 / hectogram
Atlantic Salmon                          3.525191572336192 / hectogram
Bison                                   2.4228802614118043 / hectogram
Bacon                                   1.7614934748571718 / hectogram
Eggs                                   0.07825593780666174 / hectogram
Turkey                                  2.8638047857815594 / hectogram
Sausage                                  1.541031212672294 / hectogram
Butter                                   2.641137900974833 / hectogram
Raw Liver                                0.659182163932784 / hectogram
Whole milk                             0.16627189797876257 / hectogram
Yogurt                                  0.5500533441512695 / hectogram
2

### Raw Vegan Diet Units & Prices

In [109]:
vegan_prices = getUnitsAndPrices("vegan")
vegan_prices

Food
Russet Potato                                    0.37258122309244307 / hectogram
Pasta                                            0.35053499687395534 / hectogram
Lemon, raw                                        1.7068965517241381 / hectogram
Lime, raw                                         1.0298507462686566 / hectogram
Nuts, pecans, halves, raw                          5.941143019793614 / hectogram
                                                              ...               
Quinoa, fat added                                 0.8355519736806859 / hectogram
Almond milk, unsweetened, plain, refrigerated    0.19235332375630568 / hectogram
Oat milk, unsweetened, plain, refrigerated       0.20467101177721655 / hectogram
WHITE SLICED MUSHROOMS, WHITE                      1.318364327865568 / hectogram
Nutritional Yeast                                  7.046953731705065 / hectogram
Name: FDC Price, Length: 66, dtype: object

### USDA Diet Units & Prices

In [110]:
usda_prices = getUnitsAndPrices("usda")
usda_prices

Food
Beef                                              2.8638047857815594 / hectogram
Pork                                              2.2024179992269266 / hectogram
Chicken, drumstick                                0.8796444261176615 / hectogram
Canned Tuna                                        0.698424446601692 / hectogram
Atlantic Salmon                                    3.525191572336192 / hectogram
                                                              ...               
Quinoa, fat added                                 0.8355519736806859 / hectogram
Almond milk, unsweetened, plain, refrigerated    0.19235332375630568 / hectogram
Oat milk, unsweetened, plain, refrigerated       0.20467101177721655 / hectogram
WHITE SLICED MUSHROOMS, WHITE                      1.318364327865568 / hectogram
Nutritional Yeast                                  7.046953731705065 / hectogram
Name: FDC Price, Length: 84, dtype: object

### Dietary Requirements

Our constraints come from US government recommendations, which is available to view at https://www.dietaryguidelines.gov/sites/default/files/2021-03/Dietary_Guidelines_for_Americans-2020-2025.pdf

In [111]:
DRI_url = "https://docs.google.com/spreadsheets/d/1y95IsQ4HKspPW3HHDtH7QMtlDA66IUsCHJLutVL-MMc/"

DRIs = read_sheets(DRI_url)

# Define *minimums*
diet_min = DRIs['diet_minimums'].set_index('Nutrition')

# Define *maximums*
diet_max = DRIs['diet_maximums'].set_index('Nutrition')

Key available for students@eep153.iam.gserviceaccount.com.


### Using `solve_subsistence_problem` to analyze diet

In [112]:
def solve_subsistence_problem(FoodNutrients,Prices,dietmin,dietmax,max_weight=None,tol=1e-6):
    """Solve Stigler's Subsistence Cost Problem.

    Inputs:
       - FoodNutrients : A pd.DataFrame with rows corresponding to foods, columns to nutrients.
       - Prices : A pd.Series of prices for different foods
       - diet_min : A pd.Series of DRIs, with index corresponding to columns of FoodNutrients,
                    describing minimum intakes.
       - diet_max : A pd.Series of DRIs, with index corresponding to columns of FoodNutrients,
                    describing maximum intakes.
       - max_weight : Maximum weight (in hectograms) allowed for diet.
       - tol : Solution values smaller than this in absolute value treated as zeros.
       
    """
    try: 
        p = Prices.apply(lambda x:x.magnitude)
    except AttributeError:  # Maybe not passing in prices with units?
        warnings.warn("Prices have no units.  BE CAREFUL!  We're assuming prices are per hectogram or deciliter!")
        p = Prices

    p = p.dropna()

    # Compile list that we have both prices and nutritional info for; drop if either missing
    use = p.index.intersection(FoodNutrients.columns)
    p = p[use]

    # Drop nutritional information for foods we don't know the price of,
    # and replace missing nutrients with zeros.
    Aall = FoodNutrients[p.index].fillna(0)

    # Drop rows of A that we don't have constraints for.
    Amin = Aall.loc[Aall.index.intersection(dietmin.index)]
    Amin = Amin.reindex(dietmin.index,axis=0)
    idx = Amin.index.to_frame()
    idx['type'] = 'min'
    #Amin.index = pd.MultiIndex.from_frame(idx)
    #dietmin.index = Amin.index
    
    Amax = Aall.loc[Aall.index.intersection(dietmax.index)]
    Amax = Amax.reindex(dietmax.index,axis=0)
    idx = Amax.index.to_frame()
    idx['type'] = 'max'
    #Amax.index = pd.MultiIndex.from_frame(idx)
    #dietmax.index = Amax.index

    # Minimum requirements involve multiplying constraint by -1 to make <=.
    A = pd.concat([Amin,
                   -Amax])

    b = pd.concat([dietmin,
                   -dietmax]) # Note sign change for max constraints

    # Make sure order of p, A, b are consistent
    A = A.reindex(p.index,axis=1)
    A = A.reindex(b.index,axis=0)

    if max_weight is not None:
        # Add up weights of foods consumed
        A.loc['Hectograms'] = -1
        b.loc['Hectograms'] = -max_weight
        
    # Now solve problem!  (Note that the linear program solver we'll use assumes
    # "less-than-or-equal" constraints.  We can switch back and forth by
    # multiplying $A$ and $b$ by $-1$.)

    result = lp(p, -A, -b, method='interior-point')

    result.A = A
    result.b = b
    
    if result.success:
        result.diet = pd.Series(result.x,index=p.index)
    else: # No feasible solution?
        warnings.warn(result.message)
        result.diet = pd.Series(result.x,index=p.index)*np.nan  

    return result

In [113]:
## This prints out the solution to the minimum diet cost problem for a given population and diet.
def getPopulationSolution(group, diet):
    
    diet = diet.lower()
    
    if diet == "meat":
        nutrients_df = Raw_Meat_FoodNutrients
        diet_prices = meat_prices
    elif diet == "vegan":
        nutrients_df = Raw_Vegan_FoodNutrients
        diet_prices = vegan_prices
    elif diet == "usda":
        nutrients_df = USDA_FoodNutrients
        diet_prices = usda_prices
    else:
        print("Diet not found, please choose from meat, vegan, or usda.")
        return
    
    tol = 1e-6

    result = solve_subsistence_problem(nutrients_df,diet_prices,diet_min[group],diet_max[group],tol=tol)

    print("Cost of diet for %s is $%4.2f per day.\n" % (group,result.fun))

    # Put back into nice series
    diet = result.diet

    print("\nDiet (in 100s of grams or milliliters):")
    print(diet[diet >= tol])  # Drop items with quantities less than precision of calculation.
    print()

    tab = pd.DataFrame({"Outcome":np.abs(result.A).dot(diet),"Recommendation":np.abs(result.b)})
    print("\nWith the following nutritional outcomes of interest:")
    print(tab)
    print()

    print("\nConstraining nutrients are:")
    excess = tab.diff(axis=1).iloc[:,1]
    print(excess.loc[np.abs(excess) < tol*100].index.tolist())

### Results

#### Raw Meat Diet Results

In [114]:
getPopulationSolution("M 19-30", "meat")
getPopulationSolution("F 19-30", "meat")

Cost of diet for M 19-30 is $16.83 per day.


Diet (in 100s of grams or milliliters):
Raw Liver                0.430678
2% milk                  2.531126
Lime, raw                2.942569
Cashews, NFS             2.842878
Pistachio nuts, NFS      1.356587
Almond butter, creamy    1.168707
dtype: float64


With the following nutritional outcomes of interest:
                                    Outcome  Recommendation
Nutrition                                                  
Energy                          3100.000000          2400.0
Protein                          118.169883            56.0
Fiber, total dietary              42.098166            33.6
Folate, DFE                      400.000000           400.0
Calcium, Ca                     1000.000000          1000.0
Carbohydrate, by difference      201.677519           130.0
Iron, Fe                          31.726653             8.0
Magnesium, Mg                   1257.499889           400.0
Niacin                            18.729

#### Raw Vegan Diet Results

In [115]:
getPopulationSolution("M 19-30", "vegan")
getPopulationSolution("F 19-30", "vegan")

Cost of diet for M 19-30 is $6.06 per day.


Diet (in 100s of grams or milliliters):
Russet Potato            1.730485
Peanut butter, creamy    2.541607
White bread              1.038012
Collards, raw            0.115433
Carrots, raw             1.025215
Avocado, raw             2.138831
Lentils, NFS             0.832502
Oranges, raw, navels     0.822748
Nutritional Yeast        0.016000
dtype: float64


With the following nutritional outcomes of interest:
                                    Outcome  Recommendation
Nutrition                                                  
Energy                          2400.000001          2400.0
Protein                          101.904170            56.0
Fiber, total dietary              64.147959            33.6
Folate, DFE                      400.000002           400.0
Calcium, Ca                     1000.000000          1000.0
Carbohydrate, by difference      228.136583           130.0
Iron, Fe                          13.243377             8.0

##### USDA Diet Results

In [116]:
getPopulationSolution("M 19-30", "usda")
getPopulationSolution("F 19-30", "usda")

Cost of diet for M 19-30 is $4.80 per day.


Diet (in 100s of grams or milliliters):
Eggs                         1.222593
Raw Liver                    0.191353
2% milk                      6.472177
Peanut butter, creamy        0.810334
Collards, raw                0.034877
Avocado, raw                 4.952972
Black beans, NFS             0.075780
Lentils, NFS                 0.514558
Peppers, bell, green, raw    0.392977
Nutritional Yeast            0.003928
dtype: float64


With the following nutritional outcomes of interest:
                                    Outcome  Recommendation
Nutrition                                                  
Energy                          3100.000000          2400.0
Protein                           85.363937            56.0
Fiber, total dietary              51.787287            33.6
Folate, DFE                      455.058988           400.0
Calcium, Ca                     1000.000000          1000.0
Carbohydrate, by difference      135.056549  

## [C] Sensitivity of Solution: Effects of Price Changes on Subsistence Diet Cost <a class="anchor" id="sensitivity"></a>

As prices change, we should expect the minimum cost diet to also change. The code below creates a graph which changes prices away from the `base’ case one food at a time, and plots changes in total diet cost.

In [117]:
def getPriceChangesGraph(diet):

    diet = diet.lower()
    if diet == "meat":
        diet_FoodNutrients = Raw_Meat_FoodNutrients
        diet_prices = meat_prices
        graphType = "Raw Meat"
    elif diet == "vegan":
        diet_FoodNutrients = Raw_Vegan_FoodNutrients
        diet_prices = vegan_prices
        graphType = "Raw Vegan"
    elif diet == "usda":
        diet_FoodNutrients = USDA_FoodNutrients
        diet_prices = usda_prices
        graphType = "USDA"
    else:
        print("Please choose correct diet.")
        
    cf.go_offline()
    group = 'M 19-30'
    tol = 1e-6
    scale = [.5,.6,.7,.8,.9,1.,1.1,1.2,1.3,1.4,1.5]
    
    cost0 = solve_subsistence_problem(diet_FoodNutrients,diet_prices,diet_min[group],diet_max[group],tol=tol).fun
    
    Price_response={}
    for s in scale:
        cost = {}
        for i,p in enumerate(diet_prices):
            my_p = diet_prices.copy()
            my_p[i] = p*s
            result = solve_subsistence_problem(diet_FoodNutrients,my_p,diet_min[group],diet_max[group],tol=tol)
            cost[diet_prices.index[i]] = np.log(result.fun/cost0)
        Price_response[np.log(s)] = cost
        
    Price_response = pd.DataFrame(Price_response).T
    Price_response.iplot(title= "Effects of Price Changes on Subsistence Diet Composition for " + graphType + " Diet",xTitle='change in log price',yTitle='change in log cost')

In [118]:
getPriceChangesGraph("meat")

In [119]:
getPriceChangesGraph("vegan")

In [120]:
getPriceChangesGraph("usda")

## Effects of Price Changes on Subsistence Diet Composition

The code below creates a graph which changes prices just for *one* food,
  and traces out the effects of this change on all the foods consumed.



In [121]:
def getEffectsofPriceChangeGraph(dietType, ReferenceGood) :
    
    dietType = dietType.lower()
    if dietType == "meat":
        diet_FoodNutrients = Raw_Meat_FoodNutrients
        diet_prices = meat_prices
        graphType = "Raw Meat"
    elif dietType == "vegan":
        diet_FoodNutrients = Raw_Vegan_FoodNutrients
        diet_prices = vegan_prices
        graphType = "Raw Vegan"
    elif dietType == "usda":
        diet_FoodNutrients = USDA_FoodNutrients
        diet_prices = usda_prices
        graphType = "USDA"
    else:
        print("Please choose correct diet.")
        
    group = 'M 19-30'
    tol = 1e-6
    
    cf.go_offline()

    scale = [0.5,0.75,0.9,1.,1.1,1.2,1.3,1.4,1.5,2,4]

    cost0 = solve_subsistence_problem(diet_FoodNutrients,diet_prices,diet_min[group],diet_max[group],tol=tol).fun

    my_p = diet_prices.copy()

    diet = {}
    for s in scale:

        my_p[ReferenceGood] = diet_prices[ReferenceGood]*s
        result = solve_subsistence_problem(diet_FoodNutrients,my_p,diet_min[group],diet_max[group],tol=tol)
        diet[my_p[ReferenceGood]] = result.diet

    Diet_response = pd.DataFrame(diet).T
    Diet_response.index.name = '%s Price' % ReferenceGood

    Diet_response.reset_index(inplace=True)

    # Get rid of units for index (cufflinks chokes)
    Diet_response['%s Price' % ReferenceGood] = Diet_response['%s Price' % ReferenceGood].apply(lambda x: x.magnitude)

    Diet_response = Diet_response.set_index('%s Price' % ReferenceGood)

    # Just look at goods consumed in quantities greater than error tolerance
    Diet_response.loc[:,(Diet_response>tol).sum()>0].iplot(title=ReferenceGood + " Pricing Change Effect on Hectograms Other Nutrients (" +graphType + ")",xTitle='%s Price' % ReferenceGood,yTitle='Hectograms')
    
    return Diet_response

In [122]:
Beef_Meat_response = getEffectsofPriceChangeGraph("meat", "Beef")

/opt/conda/lib/python3.9/site-packages/pandas/core/dtypes/cast.py:1990: UnitStrippedWarning:

The unit of the quantity is stripped when downcasting to ndarray.



In [123]:
Beef_USDA_response = getEffectsofPriceChangeGraph("usda", "Beef")

In [124]:
Carrots_Vegan_response = getEffectsofPriceChangeGraph("vegan", "Carrots")

## Effects of Price Changes on Subsistence Diet Nutrition

The code below creates a graph which uses the food price changes
  described above, but maps into nutrients.

In [125]:
def getPriceChangeNutrientMapGraph(dietType, Diet_response, ReferenceGood):
    
    dietType = dietType.lower()
    if dietType == "meat":
        diet_FoodNutrients = Raw_Meat_FoodNutrients
        diet_prices = meat_prices
        graphType = "Raw Meat"
    elif dietType == "vegan":
        diet_FoodNutrients = Raw_Vegan_FoodNutrients
        diet_prices = vegan_prices
        graphType = "Raw Vegan"
    elif dietType == "usda":
        diet_FoodNutrients = USDA_FoodNutrients
        diet_prices = usda_prices
        graphType = "USDA"
    else:
        print("Please choose correct diet.")
    
    
    # Matrix product maps quantities of food into quantities of nutrients
    NutrientResponse = Diet_response@diet_FoodNutrients.T

    # Drop columns of missing nutrients
    NutrientResponse = NutrientResponse.loc[:,NutrientResponse.count()>0]
    NutrientResponse.iplot(title="Effects of Price Changes on Subsistence Diet Nutrition for " + ReferenceGood + " (" + graphType + ")", xTitle='%s Price' % ReferenceGood,yTitle='Nutrients')

In [126]:
getPriceChangeNutrientMapGraph("meat", Beef_Meat_response, "Beef")

In [127]:
getPriceChangeNutrientMapGraph("usda", Beef_USDA_response, "Beef")

In [128]:
getPriceChangeNutrientMapGraph("vegan", Carrots_Vegan_response, "Carrot")

## Adding Constraint on Total Weight



At least at some prices the minimum cost subistence diet involves
eating unreasonable amounts of food (e.g., 10 kilograms of cabbage per
day).  We can easily add an additional constraint of the form
$$
     \sum x_i \leq \text{max weight}
$$
to our linear programming problem since it&rsquo;s just another linear
inequality, and this may give us more realistic results.



### Price Changes and Subsistence Diet Composition with Weight Constraint



Re-do our analysis of changing prices, but with a constraint that
  total weight of diet must be less that 12 hectograms (1.2 kg).



In [129]:
def getEffectsofPriceChangeGraphConstraint(dietType, ReferenceGood) :
    
    dietType = dietType.lower()
    if dietType == "meat":
        diet_FoodNutrients = Raw_Meat_FoodNutrients
        diet_prices = meat_prices
        graphType = "Raw Meat"
    elif dietType == "vegan":
        diet_FoodNutrients = Raw_Vegan_FoodNutrients
        diet_prices = vegan_prices
        graphType = "Raw Vegan"
    elif dietType == "usda":
        diet_FoodNutrients = USDA_FoodNutrients
        diet_prices = usda_prices
        graphType = "USDA"
    else:
        print("Please choose correct diet.")
        
    group = 'M 19-30'
    tol = 1e-6
    
    cf.go_offline()

    scale = [0.5,0.75,0.9,1.,1.1,1.2,1.3,1.4,1.5,2,4]

    cost0 = solve_subsistence_problem(diet_FoodNutrients,diet_prices,diet_min[group],diet_max[group],max_weight=12,tol=tol).fun

    my_p = diet_prices.copy()

    diet = {}
    for s in scale:

        my_p[ReferenceGood] = diet_prices[ReferenceGood]*s
        result = solve_subsistence_problem(diet_FoodNutrients,my_p,diet_min[group],diet_max[group],max_weight=12,tol=tol)
        diet[my_p[ReferenceGood]] = result.diet

    Diet_response = pd.DataFrame(diet).T
    Diet_response.index.name = '%s Price' % ReferenceGood

    Diet_response.reset_index(inplace=True)

    # Get rid of units for index (cufflinks chokes)
    Diet_response['%s Price' % ReferenceGood] = Diet_response['%s Price' % ReferenceGood].apply(lambda x: x.magnitude)

    Diet_response = Diet_response.set_index('%s Price' % ReferenceGood)

    # Just look at goods consumed in quantities greater than error tolerance
    Diet_response.loc[:,(Diet_response>tol).sum()>0].iplot(title=ReferenceGood + " Price Changes and Subsistence Diet Composition with Weight Constraint (" +graphType + ")",xTitle='%s Price' % ReferenceGood,yTitle='Hectograms')
    
    return Diet_response

In [130]:
Beef_Meat_response_Constraint = getEffectsofPriceChangeGraphConstraint("meat", "Beef")

In [131]:
Beef_Meat_response_Constraint = getEffectsofPriceChangeGraphConstraint("vegan", "Carrots")

In [132]:
Beef_Meat_response_Constraint = getEffectsofPriceChangeGraphConstraint("usda", "Beef")